## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.55,79,97,14.83,moderate rain
1,1,Bambous Virieux,MU,-20.3428,57.7575,82.65,65,40,13.80,scattered clouds
2,2,Lompoc,US,34.6391,-120.4579,58.12,94,1,13.80,mist
3,3,Kapaa,US,22.0752,-159.3190,75.52,75,20,1.01,heavy intensity rain
4,4,Ereymentau,KZ,51.6199,73.1033,30.51,95,100,9.66,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 98


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.55,79,97,14.83,moderate rain
1,1,Bambous Virieux,MU,-20.3428,57.7575,82.65,65,40,13.80,scattered clouds
3,3,Kapaa,US,22.0752,-159.3190,75.52,75,20,1.01,heavy intensity rain
7,7,New Norfolk,AU,-42.7826,147.0587,72.90,47,92,5.99,overcast clouds
8,8,Bhainsdehi,IN,21.6467,77.6325,75.88,37,0,3.62,clear sky
10,10,Rikitea,PF,-23.1203,-134.9692,76.73,75,97,7.96,overcast clouds
12,12,Itarema,BR,-2.9248,-39.9167,80.80,80,53,15.52,broken clouds
16,16,Georgetown,MY,5.4112,100.3354,84.13,71,20,10.36,few clouds
19,19,Atuona,PF,-9.8000,-139.0333,78.69,77,4,18.25,clear sky
22,22,Kahului,US,20.8947,-156.4700,74.84,94,75,25.32,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                255
City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                255
City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.55,moderate rain,3.0707,172.7902,
1,Bambous Virieux,MU,82.65,scattered clouds,-20.3428,57.7575,
3,Kapaa,US,75.52,heavy intensity rain,22.0752,-159.3190,
7,New Norfolk,AU,72.90,overcast clouds,-42.7826,147.0587,
8,Bhainsdehi,IN,75.88,clear sky,21.6467,77.6325,
10,Rikitea,PF,76.73,overcast clouds,-23.1203,-134.9692,
12,Itarema,BR,80.80,broken clouds,-2.9248,-39.9167,
16,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
19,Atuona,PF,78.69,clear sky,-9.8000,-139.0333,
22,Kahului,US,74.84,heavy intensity rain,20.8947,-156.4700,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
    
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"] + " found, near " + row["City"])
    except:
        print("Hotel not found... skipping.")
        

Isles Sunset Lodge found, near Butaritari
Casa Tia Villa found, near Bambous Virieux
Sheraton Kauai Coconut Beach Resort found, near Kapaa
The Shingles Riverside Cottages found, near New Norfolk
Prince Rathore HOME found, near Bhainsdehi
People ThankYou found, near Rikitea
Sea Lounge! Bar, Club & Restaurant found, near Itarema
Cititel Penang found, near Georgetown
Villa Enata found, near Atuona
Maui Seaside Hotel found, near Kahului
Welcomhotel By ITC Hotels, Bay Island, Port Blair found, near Port Blair
Paradise Inn found, near Avarua
Observatory Guest House found, near Busselton
K G N HOTEL found, near Dhanwar
Molika Springs Motel found, near Moree
Grage Hotel Bengkulu found, near Bengkulu
Pousada Porto Praia found, near Arraial Do Cabo
Valley Breeze Guest House found, near Grenville
INN NEW HORIZON found, near Touros
Hotel Kou-Bugny found, near Vao
Shanti Maurice Resort & Spa found, near Souillac
Legend Hotel found, near Lao Cai
Tarumã Tropical Hotel found, near Conceicao Do Araguai

Posada Paraiso found, near Kantunilkin
WFP Guesthouse found, near Bria
Ramah Kost KITA found, near Pemangkat
Hotel palance found, near Puerto Quijarro
Nuli Sapi found, near Samarai
Balandra Hotel found, near Manta
Majestic Oasis Apartments found, near Port Augusta
Airport Lodge Kiunga found, near Kiunga
Baliraya found, near Cavinti
The Gem Hotel Beaufort found, near Beaufort
Hotel Gilmart found, near Ortega
Medhufushi Island Resort found, near Muli
OYO Hotel Iramar found, near Aracruz
Predio Livraria Ahmed found, near Chimoio
Maunacaribe found, near Maunabo
Lidera found, near Carutapera
Hotel not found... skipping.
Cititel Penang found, near Butterworth
Mercure Padang found, near Padang
Hostel khaga sardai road found, near Khaga
Gîte kajeon found, near Poum
Hotel Kontiki found, near Sao Francisco Do Sul
Aipana Plaza Hotel found, near Boa Vista
Sangalad Lodging found, near Ron
Tour Eiffel Hotel Benin found, near Porto Novo
Atlantic Sand Oasis found, near Arecibo
Nam Long Hotel found, ne

In [11]:
# Verify if there are any empty rows.
hotel_df.count()

City                   255
Country                255
Max Temp               255
Current Description    255
Lat                    255
Lng                    255
Hotel Name             255
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.55,moderate rain,3.0707,172.7902,Isles Sunset Lodge
1,Bambous Virieux,MU,82.65,scattered clouds,-20.3428,57.7575,Casa Tia Villa
3,Kapaa,US,75.52,heavy intensity rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,New Norfolk,AU,72.90,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
8,Bhainsdehi,IN,75.88,clear sky,21.6467,77.6325,Prince Rathore HOME
10,Rikitea,PF,76.73,overcast clouds,-23.1203,-134.9692,People ThankYou
12,Itarema,BR,80.80,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
16,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Cititel Penang
19,Atuona,PF,78.69,clear sky,-9.8000,-139.0333,Villa Enata
22,Kahului,US,74.84,heavy intensity rain,20.8947,-156.4700,Maui Seaside Hotel


In [13]:
# Verify that the cleaned Dataframe has a relevant sample size
clean_hotel_df.count()


City                   240
Country                240
Max Temp               240
Current Description    240
Lat                    240
Lng                    240
Hotel Name             240
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(25.0, 30.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))